### IPMOF - InterPenetrating Metal Organic Frameworks

#### Initialize necessary information

1. Read FF_Parameters excel file to get force field parameters for atoms
2. Initialize force field selection, cut_off radius and grid size for energy map

#### Read structural information for MOF files in a given directory
1. Read MOF files in ".mol2" format from given directory and create a list
2. Create MOF objects for structure files
3. Initialize structural information for the MOFs

#### Read simulation input parameters
1. Read simulation parameters from input file

#### Calculate energy map
1. Determine packing amount of the MOF
2. Calculate packed coordinates for the base MOF
3. Calculate energy map

#### Start interpenetration
1. Energy map + mobile_mof

In [1]:
import math
import os
#os.chdir(r'/home/kutay/Documents/git/IPMOF/IPMOF_Python')
os.chdir(r'C:\Kutay\IPMOF\IPMOF_Python')
from forcefield import *
from crystal import *
from energymap import *
from visualize import *
from interpenetration import *
from quaternion import *
%pylab inline

# Read excel file containing force field information
#excel_file_path = '/home/kutay/Documents/Research/FF_Parameters.xlsx'
excel_file_path = r'C:\Users\kutay\iPython\IPMOF\FF_Parameters.xlsx'
uff = read_ff_parameters(excel_file_path, 'uff')

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


### File Input Options
- <b> Built-in read_mol2 </b>
<pre>
uc_size, uc_angle, atom_names, atom_coors = read_mol2(mol2_path)
mof = MOF()
mof.initialize(mof.mol2_path)
</pre> 
- <b> Ase pdb read (cif read gives error) </b>
<pre>
from ase.io import read
mof_atoms = read(mof_dir, format='pdb')
mof_atoms.get_positions()                # Coordinates
mof_atoms.get_chemical_symbols()         # Atom names
mof_atoms.get_cell()                     # Cell vectors? (check)
mof_obj.get_number_of_atoms()            # Num of atoms
mof_obj.get_volume()                     # Unit cell volume

</pre>
- <b> Open babel </b>
<pre>
babel -icif *.cif -opdb *.pdb
</pre>

In [2]:
# Create list of MOFs
#mol2_dir = r'/home/kutay/Documents/Research/MOFs/IPMOF_Python/mol2'
mol2_dir = r'C:\Kutay\MOFs\IPMOF_Python'
mol2_list = get_mof_list(mol2_dir, '.mol2')
print(mol2_list)

['KINFAQ.mol2', 'NUVWIL.mol2', 'OFODAP_clean.mol2', 'QIGBIR_clean.mol2', 'UNIGEE_clean.mol2', 'ZIF-90.mol2']


In [3]:
# Read mol2 files and initialize MOF objects
base_mof = MOF()
base_mof.mol2_path = os.path.join(mol2_dir, mol2_list[2])
base_mof.initialize()
base_mof.initialize_ff(uff)
print('Base MOF selected as: ', base_mof.name)

mobile_mof = MOF()
mobile_mof.mol2_path = os.path.join(mol2_dir, mol2_list[4])
mobile_mof.initialize()
mobile_mof.initialize_ff(uff)
print('Mobile MOF selected as: ', mobile_mof.name)

Base MOF selected as:  OFODAP_clean
Mobile MOF selected as:  UNIGEE_clean


In [4]:
cut_off = 12

base_mof.packing_factor = Packing.factor(base_mof.uc_size, cut_off)
uc_vectors = Packing.uc_vectors(base_mof.uc_size, base_mof.uc_angle)
trans_vec = Packing.translation_vectors(base_mof.packing_factor, uc_vectors)
base_mof.packed_coors = Packing.uc_coors(trans_vec, base_mof.packing_factor, uc_vectors, base_mof.atom_coors)
base_mof.edge_points = Packing.edge_points(uc_vectors)

print('Base MOF unit cell: ', base_mof.uc_size)
print('Packing factor:', base_mof.packing_factor)
print('Num of coor :', len(base_mof.packed_coors)*len(base_mof.packed_coors[0]))

Base MOF unit cell:  [12.8597, 15.1207, 23.516]
Packing factor: [3, 3, 3]
Num of coor : 3672


In [5]:
atom_list = get_uniq_atom_list([mobile_mof])
emap = energy_map(base_mof, atom_list, cut_off, 1)

In [6]:
emap_max = [emap[-1][0], emap[-1][1], emap[-1][2]]
emap_min = [emap[0][0], emap[0][1], emap[0][2]]

coor = [1.89, 11, 4.78]
emap_index = energy_map_index(coor, emap_max, emap_min)
atom_index = energy_map_atom_index('C', atom_list)

print('Energy map energy at ', emap[emap_index][:3], ' : ', emap[emap_index][atom_index] )
interpolation_energy = trilinear_interpolate(coor, atom_index, emap, emap_max, emap_min)
print('Interpolation energy at ', coor, ' : ', interpolation_energy)

Energy map energy at  [  2.  11.   5.]  :  -591.23455496
Interpolation energy at  [1.89, 11, 4.78]  :  -230.999666661


In [7]:
# Initialize simulation parameters
structure_energy_limit = 3E12
atom_energy_limit = 3E10
rotation_limit = 20
rotation_freedom = 30

Quat = Quaternion([0, 1, 1, 1])

initial_coors = initial_coordinates(base_mof, emap, atom_list, 3E10)
trial_limit = len(initial_coors) * rotation_limit
#trial_limit = 1000
#omitted_coordinates = len(emap) - len(initial_coors)

abort_ip = False
structure_count = 0
structure_total_energy = 0
initial_coor_index = 0
trial_count = 0
percent_complete = 0
div = round(trial_limit / 19)
new_structures = []
summary = []

In [8]:
# Main interpenetration algorithm

for t in range(trial_limit):  # Can iterate over something else???
    abort_ip = False
    # Interpenetration trial loop
    # Try interpenetration for a specific orientation by going through each atom in mobile mof
    for idx in range(len(mobile_mof)):    # Can iterate over something else???
    
        if not abort_ip:
            # If the interpenetration is just starting select rotation angles
            if idx == 0:
                if trial_count % rotation_limit == 0:
                    first_point = initial_coors[initial_coor_index]
                    initial_coor_index += 1
                    #initial_coor_trial_count += 1

                # Determine random angles for rotation in 3D space
                x_angle = 2 * pi * math.floor(rand() * (360/rotation_freedom)) / (360/rotation_freedom)
                y_angle = 2 * pi * math.floor(rand() * (360/rotation_freedom)) / (360/rotation_freedom)
                z_angle = 2 * pi * math.floor(rand() * (360/rotation_freedom)) / (360/rotation_freedom)

                # Rotate first atom of the mobile MOF
                atom_name = mobile_mof.atom_names[idx]
                new_coor = Coor(mobile_mof.atom_coors[idx])
                Q = Quaternion([1, new_coor.x, new_coor.y, new_coor.z])  # Might be a better way to do this
                Q = Quat.rotation(Q.xyz(), [0, 0, 0], [1, 0, 0], x_angle)
                Q = Quat.rotation(Q.xyz(), [0, 0, 0], [0, 1, 0], y_angle)
                Q = Quat.rotation(Q.xyz(), [0, 0, 0], [0, 0, 1], z_angle)
                new_coor = Q.coor()
                trial_count += 1

                translation_vector = first_point - new_coor # Check if operation is correct

                new_structure = []
                new_structure.append([first_point.x, first_point.y, first_point.z, atom_name]) # why first point but not new coor???


            # If interpenetration is still going on
            if idx < len(base_mof) and idx > 0:
                atom_name = atom_name = mobile_mof.atom_names[idx]
                new_coor = Coor(mobile_mof.atom_coors[idx])
                Q = Quaternion([1, new_coor.x, new_coor.y, new_coor.z])  # Might be a better way to do this
                Q = Quat.rotation(Q.xyz(), [0, 0, 0], [1, 0, 0], x_angle)
                Q = Quat.rotation(Q.xyz(), [0, 0, 0], [0, 1, 0], y_angle)
                Q = Quat.rotation(Q.xyz(), [0, 0, 0], [0, 0, 1], z_angle)
                new_coor = Q.coor()

                new_coor += translation_vector
                pbc_coor = new_coor.pbc(base_mof.uc_size, base_mof.uc_angle, base_mof.frac_ucv)

                emap_index = energy_map_index(pbc_coor.xyz(), emap_max, emap_min)
                emap_atom_index = energy_map_atom_index(atom_name, atom_list)

                point_energy = trilinear_interpolate(pbc_coor.xyz(), emap_atom_index, emap, emap_max, emap_min)
                structure_total_energy += point_energy

                if structure_total_energy > structure_energy_limit:
                    structure_total_energy = 0
                    abort_ip = True
                    break  # Fix this part (break interpenetration trial loop)
                elif point_energy > atom_energy_limit:
                    structure_total_energy = 0
                    abort_ip = True
                    break  # Fix this part (break interpenetration trial loop)
                else:
                    new_structure.append([new_coor.x, new_coor.y, new_coor.z, atom_name])
                    # new_structure.append(pbc_coor.x, pbc_coor.y, pbc_coor.z, atom_name)

            # If interpenetration trial ended with no collision
            if idx == len(mobile_mof) - 1:
                print('Interpenetration found!')
                new_structures.append(new_structure)
                # Record structure information!!!!
                structure_count += 1
                structure_total_energy = 0

            # Record simulation progress according to division (div)
            if t % div == 0:
                percent_complete = round(t / trial_limit * 100);
                summary.append([percent_complete, structure_count, trial_count])
                # Record summary information


Interpenetration found!
Interpenetration found!
Interpenetration found!
Interpenetration found!
Interpenetration found!
Interpenetration found!
Interpenetration found!
Interpenetration found!
Interpenetration found!
Interpenetration found!
Interpenetration found!
Interpenetration found!
Interpenetration found!
Interpenetration found!
Interpenetration found!
Interpenetration found!
Interpenetration found!
Interpenetration found!
Interpenetration found!


In [12]:
len(new_structures[0])
# add rotate unit cell to rotate unit cell edges and pack new structure coordinates with new unit cell parameters
# might not work due to cif convention used in Packing functions
# or just rotate each atom in packed coordinates with rotation information from new structures

106

In [14]:
len(mobile_mof)

106